In [1]:
import numpy as np
import pandas as pd

import sqlite3 as sql

In [2]:
db = sql.connect('northwind.db')

In [3]:
# query di bawah akan menampilkan tabel - tabel yang ada di database kita
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

In [4]:
# buat variabel cursor dari koneksi ke database kita
cursor = db.cursor()

In [5]:
# menjalankan query dan menampilkan hasil
cursor.execute(query)
print(cursor.fetchall())

[('Suppliers',), ('Shippers',), ('Employees',), ('Customers',), ('Orders',), ('Categories',), ('Products',), ('OrderDetails',)]


### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

In [8]:
pd.read_sql("SELECT * FROM orders LIMIT 3;", db)

,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2014-07-04 08:00:00.000,2014-08-01 00:00:00.000,2014-07-16 00:00:00.000,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,2014-07-05 04:00:00.000,2014-08-16 00:00:00.000,2014-07-10 00:00:00.000,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,2014-07-08 15:00:00.000,2014-08-05 00:00:00.000,2014-07-12 00:00:00.000,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil


In [9]:
pd.read_sql("SELECT COUNT(*) FROM suppliers;", db)

,COUNT(*)
0,29


In [10]:
pd.read_sql("SELECT UPPER(City), * FROM Suppliers LIMIT 3;", db)

,UPPER(City),SupplierID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,LONDON,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,None,EC1 4SD,UK,(171) 555-2222,None,None
1,NEW ORLEANS,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,None,#CAJUN.HTM#
2,ANN ARBOR,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,None


### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

In [11]:
pd.read_sql("SELECT AVG(freight) FROM Orders;", db)

,AVG(freight)
0,80.627205


In [16]:
pd.read_sql("SELECT AVG(freight), ShipCountry FROM Orders GROUP BY 2;", db)

,AVG(freight),ShipCountry
0,37.411250,Argentina
1,184.787500,Austria
2,67.375789,Belgium
3,58.797470,Brazil
4,73.269667,Canada
5,77.566111,Denmark
6,41.404091,Finland
7,80.723766,France
8,92.485902,Germany
9,145.012632,Ireland


In [17]:
pd.read_sql("SELECT AVG(freight), ShipCountry, ShipName FROM Orders GROUP BY 2, 3;", db)

,AVG(freight),ShipCountry,ShipName
0,12.126667,Argentina,Cactus Comidas para llevar
1,61.328000,Argentina,Océano Atlántico Ltda.
2,43.836000,Argentina,Rancho grande
3,206.846333,Austria,Ernst Handel
4,118.611000,Austria,Piccolo und mehr
...,...,...,...
85,96.647143,USA,White Clover Markets
86,33.900000,Venezuela,GROSELLA-Restaurante
87,69.953333,Venezuela,HILARION-Abastos
88,52.457857,Venezuela,LILA-Supermercado


In [ ]:
pd.read_sql("SELECT AVG(freight), ShipCountry, COUNT(DISTINCT ShipName) FROM Orders GROUP BY 2;", db)

,AVG(freight),ShipCountry,COUNT(DISTINCT ShipName)
0,37.411250,Argentina,3
1,184.787500,Austria,2
2,67.375789,Belgium,2
3,58.797470,Brazil,9
4,73.269667,Canada,3
5,77.566111,Denmark,2
6,41.404091,Finland,2
7,80.723766,France,10
8,92.485902,Germany,12
9,145.012632,Ireland,1


In [ ]:
query_1 = """
SELECT AVG(freight)
      , MAX(freight)
      , MIN(freight)
      , ShipCountry
      , COUNT(DISTINCT ShipName)
      FROM Orders GROUP BY 4
"""
pd.read_sql_query(query_1, db)

,AVG(freight),MAX(freight),MIN(freight),ShipCountry,COUNT(DISTINCT ShipName)
0,37.411250,217.86,0.33,Argentina,3
1,184.787500,789.95,5.29,Austria,2
2,67.375789,424.30,0.17,Belgium,2
3,58.797470,890.78,0.14,Brazil,9
4,73.269667,379.13,0.94,Canada,3
5,77.566111,299.09,2.92,Denmark,2
6,41.404091,180.45,0.59,Finland,2
7,80.723766,2000.00,0.02,France,10
8,92.485902,1007.64,0.15,Germany,12
9,145.012632,603.54,16.74,Ireland,1


### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

In [ ]:
# It's an error to use WHERE to put a condition on an aggregated column.
pd.read_sql("SELECT AVG(freight) AS AVG_freight, ShipCountry FROM Orders GROUP BY 2 WHERE AVG_freight > 70;", db)

DatabaseError: ignored

In [ ]:
pd.read_sql("SELECT AVG(freight) AS AVG_freight, ShipCountry FROM Orders GROUP BY 2 HAVING AVG_freight > 70;", db)

,AVG_freight,ShipCountry
0,184.787500,Austria
1,73.269667,Canada
2,77.566111,Denmark
3,80.723766,France
4,92.485902,Germany
5,145.012632,Ireland
6,87.502703,Sweden
7,76.029444,Switzerland
8,112.879426,USA


In [18]:
query_1 = """
SELECT AVG(freight) as AVG_freight
      , MAX(freight) as MAX_freight
      , MIN(freight) as MIN_freight
      , ShipCountry
      , COUNT(DISTINCT ShipName) as COUNT_shipname
      FROM Orders
      GROUP BY 4
      HAVING AVG_freight > 50
      AND COUNT_shipname > 2
"""
pd.read_sql_query(query_1, db)

,AVG_freight,MAX_freight,MIN_freight,ShipCountry,COUNT_shipname
0,58.797470,890.78,0.14,Brazil,9
1,73.269667,379.13,0.94,Canada,3
2,80.723766,2000.00,0.02,France,10
3,92.485902,1007.64,0.15,Germany,12
4,52.754821,288.43,0.90,UK,7
5,112.879426,830.75,0.20,USA,13
6,59.460435,210.19,0.12,Venezuela,4


# Mini Case

Selamat ! Kamu baru saja diterima bekerja sebagai seorang Jr. Data Analyst di perusahaan Nortwind Traders!. Hari ini adalah hari pertama kamu bekerja, dan manager kamu ingin kamu secara perlahan mengetahui seluk beluk dari perusahaan tempat kamu bekerja. Kamu diminta untuk mengerjakan beberapa tugas berikut menggunakan bantuan SQL yang sudah kamu pelajari sebelumnya.
Tugas kamu adalah membuat query untuk menjawab beberapa tugas yang akan diberikan di bawah ini, kemudian kamu dapat memberikan komentar sesuai dengan hasil query kamu.
Selamat bekerja!!

### 6.1. Menganalisa aktivitas pemesanan

In [ ]:
# tampilkan data pemesanan pada customer id VINET dengan tujuan negara France, dan memiliki nilai maksimal Freight lebih dari 10

### 6.2 Mengidentifikasi Customer

In [ ]:
# tampilkan data jumlah customer di masing-masing country yang tinggal di Perancis dan USA